# Valideren van alle karteringen
Maar voor nu enkel groningen

In [1]:
import geopandas as gpd
import pandas as pd

from veg2hab import validation

import geopandas as gpd

pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)

omgezette_karteringen_prefix = "../testing/omgezette_vegetatiekarteringen/"
habitatkarteringen_prefix = "../testing/"

/home/jordydelange/.cache/pypoetry/virtualenvs/veg2hab-CuqoUkZb-py3.7/lib/python3.7/site-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.11.2-CAPI-1.17.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


### Groningen

In [2]:
validationinfo = pd.read_excel(
    "../data/validationinfo.xlsx", 
    engine='openpyxl', 
    usecols=[
        "path_pred",
        "naam_in_overzicht_habkart",
        "provincie",
        "path_true",
        "habtype_cols",
        "percentage_cols",
        "werkt_nu",
    ]
)
# Omzetten van pd.NA naar None om te voorkomen dat we overal "if not pd.isna()" moeten doen
validationinfo = validationinfo.applymap(lambda x: None if pd.isna(x) else x)

# Uitfilteren van karteringen die we om wat voor reden dan ook niet goed om kunnen zetten (zie kolom "reden dat hij niet werkt" in de excel)
validationinfo = validationinfo[validationinfo.werkt_nu]

# Voor nu nog even enkel groningen
validationinfo = validationinfo[validationinfo["provincie"] == "Groningen"]
validationinfo["path_true"] = validationinfo["path_true"].apply(lambda x: "../testing/" + x[2:])
validationinfo["path_pred"] = validationinfo["path_pred"].apply(lambda x: omgezette_karteringen_prefix + x[2:])
validationinfo

# Load in the habitatkarteringen
trues = {}
preds = {}
for row in validationinfo.itertuples():
    # We printen even wat dingen zodat je kunt zien waar errors vandaan komen (als die er zijn)
    print(f"----- {row.naam_in_overzicht_habkart}")
    print("--- trues")
    # Habitatkarteringen (trues)
    raw = gpd.read_file(row.path_true)
    parsed = validation.parse_habitat_percentages(raw, habtype_cols=row.habtype_cols, percentage_cols=row.percentage_cols, how_to_handle_missing_percentages="split_equally")
    trues[row.naam_in_overzicht_habkart] = parsed
    print("--- preds")
    # Door ons omgezette vegetatiekarteringen (preds)
    raw = gpd.read_file(row.path_pred)
    parsed = validation.parse_habitat_percentages(raw)
    preds[row.naam_in_overzicht_habkart] = parsed



----- SBB Westerwolde 2020
--- trues
--- preds
----- SBB ZWK 2010 0814_Tussen de Gasten 2010
--- trues
--- preds
----- SBB ZWK 2010 0815_Trimunt_2010
--- trues
--- preds
----- GL_Zuidlaardermeer2019
--- trues
--- preds
----- SGL Hunzedal en Leekstermeer2021 Leekstermeer 2021
--- trues


/home/jordydelange/git/veg2hab/veg2hab/validation.py:81: UserWarning: No non-null habitat types found, returning 100% of H0000
  warnings.warn(f"No non-null habitat types found, returning 100% of H0000")


--- preds
----- SBB Lauwersmeer 2015
--- trues
--- preds
----- SGL Hunzedal en Leekstermeer2021 Hunzedal
--- trues
--- preds


### Combineren van de preds en de trues

Bij het over elkaar heen leggen van de vlakken in preds en die in trues ontstaan er soms heel veel kleine slivervlakken; deze worden verwijderd door alle vlakken kleiner dan 1m² te droppen. Of en hoeveel vlakken er gedropt worden wordt hier beneden gerapporteerd. Als hier grote getallen uitkomen (ruwweg 1 m² per 1000 rows dropped is normaal) is er wellicht wat mis gegaan.

In [3]:
for key in trues.keys():
    assert key in preds.keys(), f"Key {key} not found in predictions"

output = {}
for key, true in trues.items():
    print(f"---- {key}")
    pred = preds[key]
    combined = validation.spatial_join(pred, true, how="intersection")
    output[key] = validation.voeg_correctheid_toe_aan_df(combined)

---- SBB Westerwolde 2020


/home/jordydelange/git/veg2hab/veg2hab/validation.py:182: UserWarning: Dropping 4193 rows based on area (presumed rounding errors) with a combined area of 2.495387607979196 m²
  f"Dropping {mask.sum()} rows based on area (presumed rounding errors) with a combined area of {overlayed[mask].area.sum()} m²"


---- SBB ZWK 2010 0814_Tussen de Gasten 2010


/home/jordydelange/git/veg2hab/veg2hab/validation.py:182: UserWarning: Dropping 8208 rows based on area (presumed rounding errors) with a combined area of 0.0 m²
  f"Dropping {mask.sum()} rows based on area (presumed rounding errors) with a combined area of {overlayed[mask].area.sum()} m²"


---- SBB ZWK 2010 0815_Trimunt_2010
---- GL_Zuidlaardermeer2019


...

/home/jordydelange/git/veg2hab/veg2hab/validation.py:182: UserWarning: Dropping 1248 rows based on area (presumed rounding errors) with a combined area of 0.7538292024627701 m²
  f"Dropping {mask.sum()} rows based on area (presumed rounding errors) with a combined area of {overlayed[mask].area.sum()} m²"


---- SBB Lauwersmeer 2015


/home/jordydelange/git/veg2hab/veg2hab/validation.py:182: UserWarning: Dropping 17516 rows based on area (presumed rounding errors) with a combined area of 17.750447674154106 m²
  f"Dropping {mask.sum()} rows based on area (presumed rounding errors) with a combined area of {overlayed[mask].area.sum()} m²"


---- SGL Hunzedal en Leekstermeer2021 Hunzedal


/home/jordydelange/git/veg2hab/veg2hab/validation.py:182: UserWarning: Dropping 8748 rows based on area (presumed rounding errors) with a combined area of 0.0 m²
  f"Dropping {mask.sum()} rows based on area (presumed rounding errors) with a combined area of {overlayed[mask].area.sum()} m²"


Confusion matrices

In [4]:
for key, gdf in output.items():
    print("------ " + key)
    print(validation.bereken_volledige_conf_matrix(gdf, method="percentage"))
    print("------------------------")

------ SBB Westerwolde 2020
true_hab   H0000  H3130  H3160  H4010A  H4010_A  H4030  H6230  H6410  H6430_A  H7110_B  H7150  H91D0  HXXXX
pred_hab                                                                                                   
H0000     492.38    0.0    0.0     0.0      0.4   2.09   9.16    0.0      0.0      0.0    0.0    0.0    0.0
H3130       0.00    0.0    0.0     0.0      0.0   0.00   0.00    0.0      0.0      0.0    0.0    0.0    0.0
H3160       0.00    0.0    0.0     0.0      0.0   0.00   0.00    0.0      0.0      0.0    0.0    0.0    0.0
H4010A      0.00    0.0    0.0     0.0      0.0   0.00   0.00    0.0      0.0      0.0    0.0    0.0    0.0
H4010_A     0.00    0.0    0.0     0.0      0.0   0.00   0.00    0.0      0.0      0.0    0.0    0.0    0.0
H4030       0.00    0.0    0.0     0.0      0.0   0.00   0.00    0.0      0.0      0.0    0.0    0.0    0.0
H6230       0.00    0.0    0.0     0.0      0.0   0.00   0.00    0.0      0.0      0.0    0.0    0.0    0.0


Saving as gpkg

In [5]:
for key, gdf in output.items():
    gdf.to_file(f"../testing/validation_output/{key}.gpkg")